In [1]:
# based on https://github.com/chkwon/TrafficAssignment.jl

include(joinpath("/home/jzh/Dropbox/Research/",
"Data-driven_estimation_inverse_optimization", 
"Experiments/InverseVIsTraffic/Julia_files/load_network_uni-class.jl"))

using Graphs

function create_graph(start_node, end_node)
    @assert Base.length(start_node)==Base.length(end_node)

    no_node = max(maximum(start_node), maximum(end_node))
    no_arc = Base.length(start_node)

    graph = simple_inclist(no_node)
    for i=1:no_arc
        add_edge!(graph, start_node[i], end_node[i])
    end
    return graph
end

function get_vector(state, origin, destination, link_dic)
    current = destination
    parent = -1
    x = zeros(Int, maximum(link_dic))

    while parent != origin
        parent = state.parents[current]

        link_idx = link_dic[parent,current]

        if link_idx != 0
            x[link_idx] = 1
        end

        current = parent
    end

    return x
end

ta_data = load_ta_network("East Massachusetts")

# unpacking data from ta_data
network_name = ta_data.network_name

number_of_zones = ta_data.number_of_zones
number_of_nodes = ta_data.number_of_nodes
first_thru_node = ta_data.first_thru_node
number_of_links = ta_data.number_of_links

start_node = ta_data.start_node
end_node = ta_data.end_node
capacity = ta_data.capacity
link_length = ta_data.link_length

free_flow_time = ta_data.free_flow_time
B = ta_data.B
power = ta_data.power
speed_limit = ta_data.speed_limit
toll = ta_data.toll
link_type = ta_data.link_type
number_of_zones = ta_data.number_of_zones
total_od_flow = ta_data.total_od_flow
travel_demand = ta_data.travel_demand
od_pairs = ta_data.od_pairs

toll_factor = ta_data.toll_factor
distance_factor = ta_data.distance_factor

best_objective = ta_data.best_objective

# preparing a graph
graph = create_graph(start_node, end_node)
link_dic = sparse(start_node, end_node, 1:number_of_links);

In [2]:
function MSA(coeffs) 
    polyEval(coeffs, pt) = sum([coeffs[i] * pt^(i-1) for i = 1:length(coeffs)]) 

    function BPR(x)
        bpr = similar(x)
        for i=1:length(bpr)
            bpr[i] = free_flow_time[i] * polyEval( coeffs, (x[i]/capacity[i]) ) 
        end
        return bpr
    end

    function all_or_nothing(travel_time)
        state = []
        path = []
        x = zeros(size(start_node))

        for r=1:size(travel_demand)[1]
            # for each origin node r, find shortest paths to all destination nodes
            state = dijkstra_shortest_paths(graph, travel_time, r)

            for s=1:size(travel_demand)[2]
                # for each destination node s, find the shortest-path vector
                # load travel demand
                x = x + travel_demand[r,s] * get_vector(state, r, s, link_dic)
            end
        end

        return x
    end

    # Finding a starting feasible solution
    travel_time = BPR(zeros(number_of_links))
    xl = all_or_nothing(travel_time)

    max_iter_no = 1e12
    l = 1
    #average_excess_cost = 1
    tol = 1e-5

    while l < max_iter_no
        l += 1

        xl_old = xl

        # Finding yl
        travel_time = BPR(xl)

        yl = all_or_nothing(travel_time)

        xl = xl + (yl - xl)/l

        xl_new = xl

        relative_gap = norm(xl_new - xl_old, 1) / norm(xl_new, 1)

        if relative_gap < tol
            break
        end
    end
    
    return xl
end

MSA (generic function with 1 method)

In [4]:
# getting the coefficients of the costs
coeffs_dict_Apr_AM = readall("../temp_files/coeffs_dict_Apr_AM.json")
coeffs_dict_Apr_AM = JSON.parse(coeffs_dict_Apr_AM)

xl = Dict{(Int64,Float64,Float64,Int64),Array{Float64,1}}()

deg_grid = [3:7]
c_grid = 1.:.5:3.
lamb_grid = [10. .^(-5:3)]

for deg in deg_grid
    for c in c_grid
        for lam in lamb_grid
            coeffs_1 = coeffs_dict_Apr_AM["($(deg),$(c),$(lam),1)"]
            coeffs_2 = coeffs_dict_Apr_AM["($(deg),$(c),$(lam),2)"]
            coeffs_3 = coeffs_dict_Apr_AM["($(deg),$(c),$(lam),3)"]
            xl[(deg, c, lam, 1)] = MSA(coeffs_1)
            xl[(deg, c, lam, 2)] = MSA(coeffs_2)
            xl[(deg, c, lam, 3)] = MSA(coeffs_3)
        end
    end
end

In [8]:
lamb_grid

9-element Array{Float64,1}:
    1.0e-5
    0.0001
    0.001 
    0.01  
    0.1   
    1.0   
   10.0   
  100.0   
 1000.0   

In [5]:
outfile = open("../temp_files/uni-class_traffic_assignment_MSA_flows_Apr_AM.json", "w")

JSON.print(outfile, xl)

close(outfile)

In [6]:
xl

Dict{(Int64,Float64,Float64,Int64),Array{Float64,1}} with 675 entries:
  (3,1.5,1.0,1)    => [1263.4863185342376,912.1085041783072,1441.9159304657621,…
  (3,1.0,0.0001,1) => [1278.6061423231524,912.102904003774,1426.7961066768473,1…
  (3,3.0,10.0,2)   => [1279.4346673646353,912.1029040040648,1425.9675816353645,…
  (4,1.0,0.01,2)   => [1253.671022092031,912.1029040035999,1451.7312269079687,1…
  (4,2.5,10.0,1)   => [1266.7723590848464,912.1029040037038,1438.6298899151534,…
  (6,1.5,1.0,3)    => [1245.7365359484297,912.1029040020902,1459.66571305157,10…
  (5,1.0,0.1,2)    => [1243.891909452202,912.1029040028566,1461.5103395477977,1…
  (4,3.0,0.001,1)  => [1288.998960068451,912.102904004478,1416.4032889315488,10…
  (5,1.5,1.0e-5,3) => [2705.4022489999998,912.1029219999999,0.0,1008.7945050000…
  (7,1.0,0.1,1)    => [1239.5401322325874,912.1207486254043,1465.9148824400318,…
  (7,1.5,100.0,2)  => [1241.718264073906,912.1208234068026,1463.736971723595,10…
  (4,1.0,10.0,3)   => [1248.0400385455

In [7]:
xl[(3,1.5,1.0,1)]

24-element Array{Float64,1}:
 1263.49 
  912.109
 1441.92 
 1008.79 
 1291.6  
 1190.37 
 1350.08 
 1153.76 
 1472.21 
 1113.8  
 1191.93 
  808.18 
  890.101
 1214.59 
 1043.35 
  933.357
 1102.06 
 1171.0  
 1133.23 
 1638.77 
  952.263
 1200.66 
 1253.44 
 1182.41 